In [1]:
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd
from anndata import AnnData
import pathlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
import seaborn as sns
import tangram as tg

sc.logging.print_header()
print(f"squidpy=={sq.__version__}")

%load_ext autoreload
%autoreload 2
%matplotlib inline

scanpy==1.9.8 anndata==0.9.2 umap==0.5.7 numpy==1.22.4 scipy==1.10.1 pandas==2.0.3 scikit-learn==1.3.2 statsmodels==0.14.1 igraph==0.11.8 pynndescent==0.5.13
squidpy==1.2.3


In [2]:
# LOAD ACTUAL DATA

adata_st = sq.datasets.visium_fluo_adata_crop()
adata_st = adata_st[
    adata_st.obs.cluster.isin([f"Cortex_{i}" for i in np.arange(1, 5)])
].copy()
img = sq.datasets.visium_fluo_image_crop()

adata_sc = sq.datasets.sc_mouse_cortex()

In [3]:
adata_sc_augmented = sc.read_h5ad("adata_sc_augmented.h5ad")
adata_st_augmented = sc.read_h5ad("adata_st_augmented.h5ad")

In [6]:
import tangram as tg
print(dir(tg))

['AnnData', 'H_sc', 'H_sc_augmented', 'H_sc_emb', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'adata_sc', 'adata_sc_augmented', 'adata_sc_embeddings', 'mpl', 'n_emb_dims', 'n_genes', 'np', 'pathlib', 'pd', 'plt', 'sc', 'skimage', 'sns', 'sq', 'tg', 'var_names']


In [8]:
import tangram as tg
#print(tg.__version__)  # This should print the version number
print(dir(tg))  # This should show the actual tangram functions

['AnnData', 'H_sc', 'H_sc_augmented', 'H_sc_emb', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'adata_sc', 'adata_sc_augmented', 'adata_sc_embeddings', 'mpl', 'n_emb_dims', 'n_genes', 'np', 'pathlib', 'pd', 'plt', 'sc', 'skimage', 'sns', 'sq', 'tg', 'var_names']


In [4]:
sc.tl.rank_genes_groups(adata_sc, groupby="cell_subclass", use_raw=False)
markers_df = pd.DataFrame(adata_sc.uns["rank_genes_groups"]["names"]).iloc[0:100, :]
markers = list(np.unique(markers_df.melt().value.values))

tg.pp_adatas(adata_sc_augmented, adata_st_augmented, genes=markers)

INFO:root:1280 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:14928 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [12]:
import tangram as tg
print(tg.__version__)

AttributeError: module 'tangram' has no attribute '__version__'

In [5]:
import tangram.mapping_utils as tm

ad_map_augmented = tm.map_cells_to_space(adata_sc_augmented, adata_st_augmented,
    mode="cells",
#     mode="clusters",
#     cluster_label='cell_subclass',  # .obs field w cell types
    density_prior='rna_count_based',
    num_epochs=500,
    # device="cuda:0",
    device='cpu',
)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 1280 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.613, KL reg: 0.001
Score: 0.733, KL reg: 0.000
Score: 0.736, KL reg: 0.000
Score: 0.737, KL reg: 0.000
Score: 0.737, KL reg: 0.000


INFO:root:Saving results..
